In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pymc3 as pm

%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
#Seismic displacement in three-dimensions, with frequency dependent amplitude, superposition of multiple waves instead of just a single one

#Defining main parameters of the model
rho0 = 1800 #Density of the homogeneous medium in kg/m^3, as taken from Beker's thesis paper, pg. 84
alpha = 0.31 #Parameter used in Beker's paper to determine primary wave speed pg. 84
beta = 0.25 #Parameter used in Beker's paper to determine primary wave speed pg 84.
nu = 0.25 #Poisson ratio as taken from Beker's thesis paper pg. 84

#Calculating the wave speeds using above paramters
CP = ((rho0/1000) / alpha)**(1.0 / beta) #Calculating primary wave speed using equation from Beker's paper, pg. 84
CS = np.sqrt((1-2*nu)/(2-2*nu)) * CP #Calculating secondary wave speed using equation from Beker's paper, pg. 84
Root = np.roots([1, -8, 8 * ((2 - nu)/(1 - nu)), -8 / (1 - nu)]) #Calculating the the ratio of the R wave speed to the p wave speed squared using equation found in Harm's and Beker's paper, pg. 20 in Beker's paper
for i in Root:
    if 0<i<1:
        CR = np.sqrt(CS**2 *i) #calculating R wave speed


#First define constants for the model
rho_0 = 1800 #density of the medium in kg/m^3
nu = 0.25 #Poisson's ratio used in Beker's paper
flist = np.array([2, 4, 6, 8, 10])
phi = 0 #phase shift of the waves
C_p = 665.17 #Primary wave speed founding using above constants and eqtns from Beker's paper, in m/s
C_s = 384.04 #Secondary wave speed in m/s
C_R = 353.05 #Rayleigh wave speed in m/s
k_p = 0.0189 #primary wave number
k_s = 0.0327 #secondary wave number
k_e = 0.0356 #horizontal wave number of rayleigh wave
q_p_z = 0.0302 #wave parameter used in Harm's paper for primary portion of vertical wave vector
q_s_z = 0.0141 #wave parameter used in Harm's paper for secondary portion of vertical wave vector
sigma = 1.464 #constant used in Harm's expression for seismic displacement
theta = np.pi / 2 #angle between wave vector and x-axis
k_e_vec = [np.cos(theta) * k_e, np.sin(theta) * k_e] #Rayleigh wave vector in x-y plane
e_k = [np.cos(theta), np.sin(theta)] #unit vector in k_e_vec direction
z = 0 #depth of medium being evaluated

x_list = np.linspace(0, 500, 50) #x-positions to be evaluated over
y_list = np.linspace(0, 500, 50) #y-positions to be evaluated over 
z_list = np.linspace(0, 500, 50) #z-positions to be evaluated over
t1_list = np.linspace(0, 6, 50) #time interval to be evaluated over

f_calc_hx = np.zeros(len(flist)) #array to place horizontal calculations at each frequency into
f_calc_hy = np.zeros(len(flist)) #array to place horizontal calculations at each frequency into
f_calc_v = np.zeros(len(flist)) #array to place vertical calculations at each frequency into

#Defining frequency dependent amplitude by estimating equation using the amplitude spectral density graph in Beker's thesis paper on pg. 100
def A(f):
    return (1.0 / 2**(-2.72)) * f**(-2.72)

#Defining function to calculate the seismic displacement vector at a single point using equation from Harm's paper "Terrestial Gravity Fluctuations" on pg. 31
def xi_sp_vect(x, y, z, t): #superposition of various Rayleigh waves at different frequencies and phases
    for i, fn in enumerate(flist):
        theta = np.random.uniform(-np.pi/2, np.pi/2) #letting each frequency have a random direction of propagation, in the positive x-direction
        omega = 2*np.pi*fn #calculating the angular frequency
        ke = omega / CR #Calculating horizontal wave number of the Rayleigh wave
        ke_vec = [np.cos(theta) * ke, np.sin(theta) * ke] #Rayleigh wave vector in x-y plane
        ks = omega / CS #Calculatin the secondary wave number of the Rayleigh wave
        kp = omega / CP #Calculating the primary wave number of the Rayleigh wave
        q_z_s = np.sqrt(ke**2 - ks**2) #Calculating wave parameter used in Harm's model, pg. 31
        q_z_p = np.sqrt(ke**2 - kp**2) #Calculating wave parameter used in Harm's model, pg. 31
        zeta = np.sqrt(q_z_p / q_z_s) #Calculating wave parameter used in Harm's model, pg. 32
        phi = np.pi / fn #letting each frequency have a different phase shift
        f_calc_hx[i] = np.cos(theta) * A(fn) * (ke * np.exp(q_z_p * z) - zeta * np.exp(q_z_s * z)) * np.sin(np.dot(ke_vec, [x,y]) - omega * t + phi) #calculating the horizontal displacement of each wave 
        f_calc_hy[i] = np.sin(theta) * A(fn) * (ke * np.exp(q_z_p * z) - zeta * np.exp(q_z_s * z)) * np.sin(np.dot(ke_vec, [x,y]) - omega * t + phi) #calculating the horizontal displacement of each wave 
        f_calc_v[i] = A(fn) * (q_z_p * np.exp(q_z_p * z) - zeta * ke * np.exp(q_z_s * z)) * np.cos(np.dot(ke_vec, [x,y]) - omega * t + phi) #calculating the vertical displacement of each wave
    xi_horiz_spx = np.sum(f_calc_hx)
    xi_horiz_spy = np.sum(f_calc_hy)
    xi_vert_sp = np.sum(f_calc_v)
    return np.array([xi_horiz_spx, xi_horiz_spy, xi_vert_sp])

In [5]:
#Seismic Newtonian Noise in three-dimensions using superposition of various Rayleigh waves

#Define constants
G = 6.67e-11 #Newton's constant of gravitation
V = (500**3) / (len(x_list) * len(y_list) * len(z_list))

grid = itertools.product(x_list, y_list, z_list) #Creating Cartesian product of the lists for each direction
gridlist = list(grid) #Creating list of the Cartesian products
points = len(x_list) * len(y_list) * len(z_list) #Calculatin the number of points in the array
nnlist = np.zeros((1,1,points,3)) #Defining list to place the Newtonian Noise calculation at each point into

#Calculating the Seismic Newtonian Noise contribution of a single point using equation (4.13) from Beker's thesis paper on pg. 92
def seisnn_sp(x, y, z, t, x0, y0, z0): #Seismic newtonian noise from a single point at one moment in time with multiple waves present
    r = [x-x0, y-y0, z-z0] #vector from point to test mass
    r_mag = np.linalg.norm(r) #magnitude of r vector
    r_hat = r / r_mag #unit vector in r direction
    xi = xi_sp_vect(x, y, z, t)
    return G * rho_0 * V * (1.0 / r_mag**3) * (xi - 3 * np.dot(r_hat, xi) * r_hat)

#Defining function to calculate total Seismic Newtonian Noise at a single point in time
def seisnnsp_total(t, x0, y0, z0): #Total seismic newtonian noise at a single moment due to multiple Rayleigh waves
    for i, p in enumerate(gridlist):
        x = gridlist[i][0] #Referencing the x-coordinate of each Cartesian product in gridlist
        y = gridlist[i][1] #Referencing the y-coordinate of each Cartesian product in gridlist
        z = gridlist[i][2] #Referencing the z-coordinate of each Cartesian product in gridlist
        nnlist[0,0,i] = seisnn_sp(x, y, z, t, x0, y0, z0)
    nntotal = np.sum(nnlist, 2)
    nntotal_mag = np.linalg.norm(nntotal)
    return nntotal_mag